# Noise Reduction Techniques for Smartphone Sleep Audio

This notebook demonstrates two classic noise reduction techniques—**Spectral Subtraction** and **Wiener Filtering**—applied to smartphone audio recorded for sleep analysis. We'll compare their effectiveness using a sample noisy audio file.

## 1. Setup and Imports

We'll use `librosa` for audio processing, `numpy` for numerical operations, `soundfile` for saving audio, and `matplotlib` for visualization.

In [ ]:
import librosa
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
import IPython.display as ipd

## 2. Load Noisy Audio

We'll load a sample noisy audio file (e.g., `sleep_demo/sleep_test_1.wav`). The sample rate is set to 16 kHz.

In [ ]:
audio_path = 'sleep_demo/sleep_test_1.wav'
target_sr = 16000
y_noisy, sr = librosa.load(audio_path, sr=target_sr)
print(f"Loaded audio: {audio_path}, Duration: {len(y_noisy)/sr:.2f}s, Sample Rate: {sr}")
ipd.Audio(y_noisy, rate=sr)

## 3. Spectral Subtraction

We estimate the noise profile from the first 0.5 seconds of the audio and subtract it from the entire signal in the frequency domain.

In [ ]:
def spectral_subtraction_auto_noise(
    y_mixed,
    sr,
    noise_duration_sec=0.5,
    n_fft=2048,
    hop_length=512,
    subtraction_factor=3.0,
    floor_factor=0.01,
    verbose=False
):
    if verbose:
        print("Starting Spectral Subtraction (Auto Noise Estimation)...")
    S_mixed_full = librosa.stft(y_mixed, n_fft=n_fft, hop_length=hop_length)
    noise_frames_count = int(np.ceil((noise_duration_sec * sr) / hop_length))
    noise_frames_count = min(noise_frames_count, S_mixed_full.shape[1])
    if noise_frames_count == 0:
        avg_power_noise_profile = np.zeros((S_mixed_full.shape[0], 1)) * 1e-10
    else:
        S_initial_noise_segment = S_mixed_full[:, :noise_frames_count]
        power_initial_noise_segment = np.abs(S_initial_noise_segment)**2
        avg_power_noise_profile = np.mean(power_initial_noise_segment, axis=1, keepdims=True)
    magnitude_mixed_full, phase_mixed_full = librosa.magphase(S_mixed_full)
    power_mixed_full = magnitude_mixed_full**2
    power_cleaned_est = power_mixed_full - subtraction_factor * avg_power_noise_profile
    spectral_floor = floor_factor * avg_power_noise_profile
    power_cleaned = np.maximum(power_cleaned_est, spectral_floor)
    magnitude_cleaned = np.sqrt(power_cleaned)
    S_cleaned = magnitude_cleaned * phase_mixed_full
    y_cleaned = librosa.istft(S_cleaned, hop_length=hop_length, length=len(y_mixed))
    return y_cleaned

In [ ]:
y_cleaned_specsub = spectral_subtraction_auto_noise(
    y_noisy, sr, noise_duration_sec=0.5, n_fft=2048, hop_length=512, subtraction_factor=3.0, floor_factor=0.01, verbose=True
)
sf.write("sleep_demo/cleaned_specsub.wav", y_cleaned_specsub, sr)
ipd.Audio(y_cleaned_specsub, rate=sr)

## 4. Wiener Filtering

We apply a basic frequency-domain Wiener filter, again estimating the noise from the initial segment.

In [ ]:
def wiener_filter_basic_auto_noise(
    y_mixed,
    sr,
    noise_duration_sec=0.5,
    n_fft=2048,
    hop_length=512,
    epsilon=1e-10,
    verbose=False
):
    if verbose:
        print("Starting Wiener Filter (Auto Noise Estimation)...")
    S_mixed_full = librosa.stft(y_mixed, n_fft=n_fft, hop_length=hop_length)
    noise_frames_count = int(np.ceil((noise_duration_sec * sr) / hop_length))
    noise_frames_count = min(noise_frames_count, S_mixed_full.shape[1])
    if noise_frames_count <= 0:
        avg_power_noise_profile = np.full((S_mixed_full.shape[0], 1), epsilon)
    else:
        S_initial_noise_segment = S_mixed_full[:, :noise_frames_count]
        power_initial_noise_segment = np.abs(S_initial_noise_segment)**2
        avg_power_noise_profile = np.mean(power_initial_noise_segment, axis=1, keepdims=True)
        avg_power_noise_profile = np.maximum(avg_power_noise_profile, epsilon)
    magnitude_mixed_full, phase_mixed_full = librosa.magphase(S_mixed_full)
    power_mixed_full = magnitude_mixed_full**2
    P_ss_estimated = power_mixed_full - avg_power_noise_profile
    P_ss_estimated = np.maximum(P_ss_estimated, 0)
    wiener_gain = P_ss_estimated / (P_ss_estimated + avg_power_noise_profile + epsilon)
    magnitude_cleaned = wiener_gain * magnitude_mixed_full
    S_cleaned = magnitude_cleaned * phase_mixed_full
    y_cleaned = librosa.istft(S_cleaned, hop_length=hop_length, length=len(y_mixed))
    return y_cleaned

In [ ]:
y_cleaned_wiener = wiener_filter_basic_auto_noise(
    y_noisy, sr, noise_duration_sec=0.75, n_fft=2048, hop_length=512, epsilon=1e-10, verbose=True
)
sf.write("sleep_demo/cleaned_wiener.wav", y_cleaned_wiener, sr)
ipd.Audio(y_cleaned_wiener, rate=sr)

## 5. Visual Comparison

Let's visualize the spectrograms of the original noisy audio, the spectral subtraction result, and the Wiener filter result.

In [ ]:
def plot_spectrograms(y_list, titles, sr, n_fft=2048, hop_length=512):
    fig, ax = plt.subplots(nrows=len(y_list), sharex=True, sharey=True, figsize=(12, 3*len(y_list)))
    if len(y_list) == 1:
        ax = [ax]
    for i, (y, title) in enumerate(zip(y_list, titles)):
        S_db = librosa.amplitude_to_db(np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length)), ref=np.max)
        librosa.display.specshow(S_db, y_axis='log', x_axis='time', sr=sr, ax=ax[i], hop_length=hop_length)
        ax[i].set(title=title)
        ax[i].label_outer()
    plt.tight_layout()
    plt.show()

plot_spectrograms(
    [y_noisy, y_cleaned_specsub, y_cleaned_wiener],
    ["Noisy Input", "Spectral Subtraction Output", "Wiener Filter Output"],
    sr
)

## 6. Listen and Compare

You can listen to the original and processed audio below to compare the effectiveness of each noise reduction technique.

In [ ]:
print("Noisy Input:")
display(ipd.Audio(y_noisy, rate=sr))
print("Spectral Subtraction Output:")
display(ipd.Audio(y_cleaned_specsub, rate=sr))
print("Wiener Filter Output:")
display(ipd.Audio(y_cleaned_wiener, rate=sr))

## 7. Conclusion

Both spectral subtraction and Wiener filtering can reduce background noise in sleep audio recordings. Spectral subtraction is simple and effective for stationary noise, while Wiener filtering can provide smoother results but may require careful parameter tuning. For best results, experiment with noise estimation duration and other parameters based on your specific audio data.